In [1]:
from sklearn import svm
import os
import random
import math
import cv2
import numpy as np

POS_SAMPLE_DIRS = ["pos_moles/cropped/"]
NEG_SAMPLE_DIRS = ["neg_moles/cropped/", "neg_caltech/cropped/"]

DATA_COLLECTION_ROOT = "../data_collection/"

POS_SAMPLE_DIRS = [DATA_COLLECTION_ROOT + d for d in POS_SAMPLE_DIRS]
NEG_SAMPLE_DIRS = [DATA_COLLECTION_ROOT + d for d in NEG_SAMPLE_DIRS]

print("Collecting samples...")

# Collect samples and sort them into positive and negative
pos_paths = []
neg_paths = []
for sample_dir in POS_SAMPLE_DIRS:
    for path in os.listdir(sample_dir):
        if path.endswith(".png"):
            pos_paths.append(sample_dir + path)

for sample_dir in NEG_SAMPLE_DIRS:
    for path in os.listdir(sample_dir):
        if path.endswith(".png"):
            neg_paths.append(sample_dir + path)
            
# Oversample positives to make up for unbalanced data
pos_neg_diff = len(neg_paths) - len(pos_paths)
repl_i = random.sample(range(0, len(pos_paths)), pos_neg_diff)
for i in repl_i:
    pos_paths.append(pos_paths[i])

# Limit sample sizes for quicker testing
pos_paths = pos_paths
neg_paths = neg_paths
        
sample_img_shape = cv2.imread(pos_paths[0]).shape
print("Image shape: %d x %d x %d" % sample_img_shape)
#sis_flat = cv2.imread(pos_paths[0]).flatten().shape

num_samples = 0

# to optimize memory, preallocate np arrays
for img_path in pos_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:
        num_samples += 1

for img_path in neg_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:
        num_samples += 1

x_size = (num_samples, sample_img_shape[0], sample_img_shape[1], sample_img_shape[2])
y_size = (num_samples)

all_x = np.zeros(x_size, dtype=np.uint8)
all_y = np.zeros(y_size, dtype=np.uint8)

sample_i = 0
for img_path in pos_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:
        all_x[sample_i] = img
        all_y[sample_i] = 1
        sample_i += 1

for img_path in neg_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:    
        all_x[sample_i] = img
        all_y[sample_i] = 0
        sample_i += 1

print("Collected %d samples!" % num_samples)

Image shape: 64 x 64 x 3
Collected 10268 samples!


In [4]:
train_dataset = []
for i in range(len(all_x)):
    img = cv2.cvtColor(all_x[i], cv2.COLOR_BGR2GRAY)
    #img = cv2.normalize(img, img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img = np.array(img).flatten()
    train_dataset.append(img)

svm_params = dict( kernel_type = cv2.ml.SVM_LINEAR, svm_type = cv2.ml.SVM_C_SVC, C=2.67, gamma=5.383 )
svm = cv2.ml.SVM_create()
#print("train dataset")
print(train_dataset[0])
#print("ys")
#print(all_y[0])
svm.train(np.float32(train_dataset), np.float32(all_y), params=svm_params)
svm.save('svm_data.dat')

[111 110 113 ...,  99  99 100]


TypeError: only length-1 arrays can be converted to Python scalars